In [1]:
#open pickle file with dataframe
import pickle
import pandas as pd
with open('../datafiles/filtered_data.pkl', 'rb') as f:
    df = pickle.load(f)
pd.set_option('display.max_columns', None)

C:\Users\case4\AppData\Local\Temp\ipykernel_23660\1571859427.py:5: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  df = pickle.load(f)


In [23]:
df.columns.to_list()

['Status/Disposal',
 'Ear Tag',
 'Ear Tag Location',
 'Brand',
 'Brand Location',
 'Tattoo',
 'Tattoo Location',
 'Other ID',
 'Other ID Location',
 'Name',
 'Registration Number',
 'Breeder',
 'Breeder Member Number',
 'Owner',
 'Owner Member Number',
 'Date of Birth',
 'Birth Method',
 'Sire',
 'Dam',
 'Status',
 'Type or Sex',
 'Percent Purebred',
 'Breed',
 'Breed 1',
 'Breed 2',
 'Breed 3',
 'Breed Comp 1',
 'Breed Comp 2',
 'Breed Comp 3',
 'Horns',
 'Color Markings',
 'EPDType',
 'EPD Reporting Period',
 'CED',
 'CED Acc',
 'BW',
 'BW Acc',
 'WW',
 'WW Acc',
 'YW',
 'YW Acc',
 'MK',
 'MK Acc',
 'TM',
 'Calving Ease Maternal EPD',
 'Calving Ease Maternal Acc',
 'Scrotal Circumference EPD',
 'Scrotal Circumference Acc',
 'Ribeye Area EPD',
 'Ribeye Area Acc',
 'Marbling EPD',
 'Marbling Acc',
 'Fat Thickness EPD',
 'Fat Thickness Acc',
 'Carcass Weight EPD',
 'Carcass Weight  Acc',
 'Percent Retail Cuts EPD',
 'Percent Retail Cuts Acc',
 'Intramuscular Fat EPD',
 'Intramuscular Fa

In [27]:
import pandas as pd
import matplotlib.pyplot as plt
import pysubgroup as ps

# Load the dataframe
df = pd.read_pickle('../datafiles/filtered_data.pkl')

# EPD columns specified by user
epd_columns = ['CED', 'BW', 'WW', 'YW', 'MK', 'TM', 'Growth']

# Bin each EPD column into three categories: Low, Medium, High
for epd in epd_columns:
    df[f'{epd}_Binned'] = pd.cut(df[epd], bins=3, labels=['Low', 'Medium', 'High'])

# Extracting dams based on 'Designation'
dams_df = df[df['Designation'] == 'Dam']

# Define the search space - consider all unique dams only
dam_selectors = [ps.EqualitySelector('Name', dam) for dam in dams_df['Name'].unique()]

# Iterate through each binned EPD column to find interesting subgroups
for epd in epd_columns:
    print(f"\nAnalyzing subgroups for EPD: {epd}")

    # Define the target variable for each binned EPD (categorical target)
    target = ps.BinaryTarget(f'{epd}_Binned', 'High')

    # Define the quality function - using WRAccQF to measure interestingness
    qf = ps.WRAccQF()  # Weighted Relative Accuracy

    # Define the search task for the current EPD, limiting to dam selectors
    task = ps.SubgroupDiscoveryTask(
        data=df,
        target=target,
        search_space=dam_selectors,
        qf=qf,
        result_set_size=10,  # Return top 10 subgroups for each EPD
        depth=1  # Depth is set to 1 to focus strictly on individual dams
    )

    # Use a subgroup discovery algorithm
    searcher = ps.SimpleDFS()  # Depth-First Search approach
    result = searcher.execute(task)

    # Convert the result to a dataframe to better understand it
    result_df = result.to_dataframe()

    # Filter out single-individual subgroups
    filtered_result_df = result_df[result_df['size_sg'] > 1]

    # Display the filtered dataframe with the best subgroups and their quality
    if filtered_result_df.empty:
        print(f"No meaningful subgroups were found for {epd}.")
    else:
        print(f"Best subgroups for {epd}:")
        print(filtered_result_df)



Analyzing subgroups for EPD: CED
No meaningful subgroups were found for CED.

Analyzing subgroups for EPD: BW
Best subgroups for BW:
   quality subgroup  size_sg  size_dataset  positives_sg  positives_dataset  \
7      0.0  Dataset      172           172            16                 16   

   size_complement  relative_size_sg  relative_size_complement  coverage_sg  \
7                0               1.0                       0.0          1.0   

   coverage_complement  target_share_sg  target_share_complement  \
7                  0.0         0.093023                      NaN   

   target_share_dataset  lift  
7              0.093023   1.0  

Analyzing subgroups for EPD: WW
Best subgroups for WW:
   quality subgroup  size_sg  size_dataset  positives_sg  positives_dataset  \
6      0.0  Dataset      172           172            39                 39   

   size_complement  relative_size_sg  relative_size_complement  coverage_sg  \
6                0               1.0                 